In [1]:
# Roman Ramirez, rr8rk@virginia.edu
# Launch 2022, Web Scraping

In [2]:
# importing modules

import numpy as np
import pandas as pd
import requests
import re

In [3]:
# a short function for getting the link to information for a given company

def get_company_link(name):
    return f'https://www.fastfoodmenuprices.com/{name}-prices/'

In [4]:
# checking to see if every link by this function produces a valid online reponse

calories = pd.read_csv('fastfood_calories.csv')
company_list = [x.lower().replace(' ','-') for x in set(calories.restaurant)]

for company in company_list:
    with requests.get(get_company_link(company)) as r:
        print(r)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [5]:
# the string to find information from the raw HTML
match_string = r'class="column-1">([\w\- &#;()–,é®\'\+]*)</td>\\n<[\w. \"=-]*>([\w. \"=-]*)</td>\\n<[\w. \"=-]*><[\w \"=-]*value="[$]([\d.]*)"'
match_string_long = r'class="column-1">([\w\- &#;()–,é®\'\+]*)</td>\\n[\<\>\w. \"=-]*([\w. \"=-]*)[\<\>\\\w. \"=-]*value="[$]([\d.]*)"'
# match_string = r'class="column-1">(?!.).*</td>\\n<[\w. \"=-]*>([\w. \"=-]*)</td>\\n<[\w. \"=-]*><[\w \"=-]*value="[$]([\d.]*)"'

# initializing the data frame by empty columns
final = pd.DataFrame(columns=['Company', 'Item', 'Size', 'Price'])

# for every company
for company in company_list:

    # find all items, sizes, and prices
    r = requests.get(get_company_link(company))
    html = str(r.content).replace(r'\xe2\x84\xa2', '')
    matcher = re.findall(match_string, html)
    matcher_long = re.findall(match_string_long, html)
    
    # adding data to big data frame
    local_frame = pd.DataFrame(matcher, columns=['Item', 'Size', 'Price'])
    local_frame_long = pd.DataFrame(matcher_long, columns=['Item', 'Size', 'Price'])    
    
    local_frame = pd.concat([local_frame, local_frame_long])
    local_frame.Item = local_frame.Item.apply(lambda x: x.replace('&#8211;', '-').replace('&amp;', '&').replace('&#8216;', '\'').replace('&#8217;', '\'').strip())
    local_frame['Company'] = [company for _ in range(len(local_frame))]
    local_frame = local_frame.reindex(columns=['Company', 'Item', 'Size', 'Price'])
    final = pd.concat([final, local_frame])

# cleaning the data frame for export
final = final.reset_index()
final.pop('index')

# exporting the data table
final.to_csv('company_item_prices.csv')
    
# displaying the final data table
final

,Company,Item,Size,Price
0,sonic,SONIC Cheeseburger,,3.89
1,sonic,SONIC Cheeseburger - Combo,,5.99
2,sonic,SuperSONIC Double Cheeseburger,,4.99
3,sonic,SuperSONIC Double Cheeseburger - Combo,,6.99
4,sonic,SuperSONIC Bacon Double Cheeseburger,,5.49
...,...,...,...,...
857,subway,Juices,,1.99
858,subway,Cheese Flatizza,,3.50
859,subway,Pepperoni Flatizza,,3.50
860,subway,Spicy Italian Flatizza,,3.50
